Import the libs

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM , Dense ,GRU
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

Load the Dataset

In [2]:
df = pd.read_csv("/content/drive/MyDrive/toxic-comments.csv")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
df.shape

(159571, 8)

In [4]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [5]:
# Define the target variables
toxicities = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]+"," ",text)
    return text

In [7]:
df["comment_text"] = df["comment_text"].apply(clean_text)

In [8]:
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i m se...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i m really not trying to edit war it s...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i can t make any real suggestions on imp...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0
5,00025465d4725e87,congratulations from me as well use the tools...,0,0,0,0,0,0
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1,1,1,0,1,0
7,00031b1e95af7921,your vandalism to the matt shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,sorry if the word nonsense was offensive to yo...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


Define i/o variables

In [9]:
# Feature and target preparation
commments = df["comment_text"].tolist()
targets = df[toxicities].values

In [10]:
targets

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [11]:
targets.shape

(159571, 6)

Prepare the Data

In [12]:
# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(commments)
sequences = tokenizer.texts_to_sequences(commments)
padded_sequences = pad_sequences(sequences, maxlen=200)

In [13]:
padded_sequences.shape

(159571, 200)

Cross Validate

In [14]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, targets, test_size=0.2, random_state=0)

In [15]:
X_train.shape

(127656, 200)

In [16]:
X_test.shape

(31915, 200)

Buulding the Model

In [17]:
# Model Definition
model = Sequential()
model.add(Embedding(5000, 128, input_length=200))
model.add(GRU(64))
model.add(Dense(6, activation='sigmoid'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense (Dense)               (None, 6)                 390       
                                                                 
Total params: 677638 (2.58 MB)
Trainable params: 677638 (2.58 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Compile the model

In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model

In [20]:
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/3
3990/3990 [==============================] - 610s 152ms/step - loss: 0.0620 - accuracy: 0.9808 - val_loss: 0.0509 - val_accuracy: 0.9922
Epoch 2/3
3990/3990 [==============================] - 611s 153ms/step - loss: 0.0459 - accuracy: 0.9860 - val_loss: 0.0480 - val_accuracy: 0.9922
Epoch 3/3
3990/3990 [==============================] - 600s 150ms/step - loss: 0.0410 - accuracy: 0.9777 - val_loss: 0.0493 - val_accuracy: 0.9770


In [21]:
new_comment = 'i will kill you, you little bitch!'

In [22]:
new_sequence = tokenizer.texts_to_sequences([clean_text(new_comment)])
padded_new_sequence = pad_sequences(new_sequence, maxlen=200)
pred = model.predict(padded_new_sequence)[0]

1/1 [==============================] - 1s 744ms/step


In [23]:
print(pred)

[0.9979919  0.5391587  0.94739574 0.92835575 0.9434511  0.10976928]


In [24]:
for toxicity, prob in zip(toxicities, pred):
    print(f"{toxicity}: {prob:.2f}")

toxic: 1.00
severe_toxic: 0.54
obscene: 0.95
threat: 0.93
insult: 0.94
identity_hate: 0.11
